<a href="https://colab.research.google.com/github/shinyanakashima/note-distillation-model/blob/main/distill_cifar10_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要
CIFAR10モデルの知識蒸留を検証する。


In [1]:
!pip install torch torchvision --quiet

In [ ]:
# 必要なデータのロード
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 1. データローダーの設定
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# CIFAR-10データセットのロード
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 2. モデルの設定
import torchvision.models as models
# 教師モデル（事前学習済みのResNet18）
teacher_model = models.resnet18(pretrained=True)
# 教師モデルは学習済みなので推論モードに
teacher_model.eval()
# CIFAR-10用に変更
# 教師モデルの最終層をCIFAR-10の10クラスに置き換える
num_ftrs = teacher_model.fc.in_features
teacher_model.fc = torch.nn.Linear(num_ftrs, 10)

# 生徒モデル（ResNet18をベースにした簡略化）
student_model = models.resnet18(pretrained=False)
student_model.fc = torch.nn.Linear(student_model.fc.in_features, 10)  # CIFAR-10用に変更

# 3. デバイス設定（GPUを使いたい場合）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

# 4. 最適化手法の設定
import torch.optim as optim
# 最適化手法（例: Adam）
optimizer = optim.Adam(student_model.parameters(), lr=0.001)

# 5. ハイパーパラメータの設定
alpha = 0.1      # ソフトターゲット損失の重み
T = 2.0          # 温度パラメータ

# 6. 学習ループ
for epoch in range(10):  # 例として10エポックでループ
    for images, labels in train_loader:  # 学習データでループ
        images, labels = images.to(device), labels.to(device)

        # 1. 教師モデルと生徒モデルの予測を取得
        with torch.no_grad():
            teacher_logits = teacher_model(images)         # 教師モデルのロジット
        student_logits = student_model(images)             # 生徒モデルのロジット

        # 2. 教師モデルのロジットからソフトターゲット確率分布を計算
        teacher_probs = F.softmax(teacher_logits / T, dim=1)

        # 3. 生徒モデルのロジットにも温度Tを適用し、ソフトマックス -> distillation損失計算
        student_log_probs = F.log_softmax(student_logits / T, dim=1)
        distill_loss = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean') * (T * T)

        # 4. 生徒モデルの通常のラベルに対するクロスエントロピー損失（ハードターゲット損失）
        hard_loss = F.cross_entropy(student_logits, labels)

        # 5. 二つの損失を合成
        loss = alpha * distill_loss + (1 - alpha) * hard_loss

        # 6. 生徒モデルのパラメータを更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


100%|██████████| 170M/170M [00:02<00:00, 71.2MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weig